In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('./sales.csv')
df.dropna(subset=['price'], inplace=True)
df.head()

df['year'] = pd.to_datetime(df.purchase_date).dt.year


FileNotFoundError: ignored

In [ ]:
import pandas as pd
import numpy as np
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
#from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np

class DataTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        df = pd.DataFrame()
        df['cost'] = X['cost'].map(self.cost2num)
        df['weight'] = X['weight'].map(self.weight2num)
        df['year'] = X['year']
        self.median = df.median(skipna=True)
        return self

    def transform(self, X):
        df = pd.DataFrame()
        df['cost'] = X['cost'].map(self.cost2num)
        df['weight'] = X['weight'].map(self.weight2num)
        df['year'] = X['year']
        df = df.fillna(self.median)
        return df

    def fit_transform(self, X, y=None):
        self.fit(X)
        return self.transform(X)

    def weight2num(self, x):
        if pd.notnull(x):
            if 'Ton' in x:
                try:
                    return float(x.split(' Ton')[0]) * 1000
                except ValueError:
                    return np.nan
            else:
                try:
                    return float(x.strip('Kg'))
                except ValueError:
                    return np.nan
        return x

    def cost2num(self, x):
        if type(x) == str:
            x = x.strip('$').strip('k')
            return float(x) * 1000
        else:
            return x


In [ ]:
class PCATransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.pca_features = ['height', 'depth', 'width']
        self.pca = PCA(n_components=3)

    def fit(self, X, y=None):
        df = pd.DataFrame()
        df['height'] = X['height'].map(self.height2num)
        df['width'] = X['width'].map(self.width2num)
        df['depth'] = X['depth'].map(self.depth2num)
        self.median = df.median(skipna=True)
        df = df.fillna(self.median)
        numeric_data_pca = df[self.pca_features]
        numeric_data_pca_scaled = StandardScaler().fit_transform(numeric_data_pca)
        self.pca.fit(numeric_data_pca_scaled)
        return self

    def transform(self, X):
        df = pd.DataFrame()
        df['height'] = X['height'].map(self.height2num)
        df['width'] = X['width'].map(self.width2num)
        df['depth'] = X['depth'].map(self.depth2num)
        df = df.fillna(self.median)
        numeric_data_pca = df[self.pca_features]
        numeric_data_pca_scaled = StandardScaler().fit_transform(numeric_data_pca)
        pca_result = self.pca.transform(numeric_data_pca_scaled)
        pca_columns = [f'PCA_{i+1}' for i in range(pca_result.shape[1])]
        pca_df = pd.DataFrame(pca_result, columns=pca_columns)
        return pca_df

    def fit_transform(self, X, y=None):
        self.fit(X)
        return self.transform(X)

    def height2num(self, x):
        if type(x) == str:
            x = x.strip('meters').strip()
            return float(x) * 100
        else:
            return x

    def width2num(self, x):
        if type(x) == str:
            x = x.strip('cm').strip()
            return float(x)
        else:
            return x

    def depth2num(self, x):
        if type(x) == str:
            x = x.strip('cm').strip()
            return float(x)
        else:
            return x


In [ ]:
def price2num(x):
    if type(x) == str:
        x = x.strip('$').replace(',', '')
    return float(x)

df['price']=df['price'].map(price2num)

In [ ]:
train_raw = df[df.year < 2015].reset_index(drop=True)
test_raw = df[df.year >= 2015].reset_index(drop=True)

numeric_features_lr = ['cost','weight', 'height', 'width', 'depth', 'year']
categorical_features_lr = ['product_level', 'product_type', 'maker', 'ingredient']
target=['price']

X_train = train_raw[numeric_features_lr]
y_train = train_raw[target]

X_test = test_raw[numeric_features_lr]
y_test = test_raw[target]


ctf=DataTransformer()
test=ctf.fit_transform(X_train)
test

,cost,weight,year
0,333000.0,3000.0,2008
1,122000.0,3000.0,1997
2,270000.0,3000.0,2001
3,97000.0,2000.0,2000
4,59000.0,2000.0,2011
...,...,...,...
2752,254000.0,3000.0,2012
2753,91000.0,2000.0,2002
2754,122000.0,2000.0,2007
2755,44000.0,2000.0,2002


In [ ]:
numeric_features = ['cost', 'weight','year']
numeric_pca = ['height','width','depth']
categorical_product_type = ['product_type']
categorical_product_level = ['product_level']
categorical_ingredient = ['ingredient']
target=['price']

X_train = train_raw[numeric_features + numeric_pca + categorical_product_type +  categorical_product_level+ categorical_ingredient]
y_train = train_raw[target]

X_test = test_raw[numeric_features + numeric_pca + categorical_product_type +  categorical_product_level+ categorical_ingredient]
y_test = test_raw[target]


In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

parameters = {
              'xgbr__gamma': [0.0, 0.1, 0.2, 0.3],
              'xgbr__learning_rate': [0.05, 0.1],
              'xgbr__n_estimators': [100, 300, 500],
              'xgbr__max_depth': [3, 5, 10],
              'xgbr__lambda': [0.5, 1, 5],
              'xgbr__min_child_weight': [3, 5, 7]
             }
scorer = make_scorer(mean_squared_error, greater_is_better = False)

In [ ]:
X_train.ingredient.nunique()

2434

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MultiLabelBinarizer

class IngredientTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # Combine training and test data to fit on both
        combined_data = pd.concat([X, y], axis=0) if y is not None else X
        self.mlb = MultiLabelBinarizer()
        # Flatten the list of ingredients and convert to string
        combined_ingredients = combined_data['ingredient'].apply(lambda x: str(x).split(',')).explode().astype(str)
        self.mlb.fit(combined_ingredients)
        return self

    def transform(self, X):
        X_transformed = X.copy()
        X_transformed['ingredient'] = X_transformed['ingredient'].astype(str).apply(lambda x: str(x).split(','))
        # Flatten the list of ingredients and convert to string
        ingredient_matrix = self.mlb.transform(X_transformed['ingredient'])
        return pd.DataFrame(ingredient_matrix, columns=self.mlb.classes_)

    def get_params(self, deep=True):
        return {}

    def set_params(self, **parameters):
        return self


In [ ]:
steps_xgbr=[
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('numeric', DataTransformer(), numeric_features),
            ('numeric_pca',PCATransformer(),numeric_pca),
            ('categorical_product_type', OneHotEncoder(handle_unknown='ignore', sparse=False), categorical_product_type),  # Produce dense matrix
            ('categorical_product_level', OrdinalEncoder(), categorical_product_level),
           ('categorical_ingredient', IngredientTransformer(), categorical_ingredient)
#            ('categorical_ingredient', OneHotEncoder(handle_unknown='ignore', sparse=False), categorical_ingredient)
        ],
#        remainder='passthrough'
         remainder='drop'
    )),
    ('xgbr', XGBRegressor(max_depth=3,learning_rate=0.05,n_estimators=100, min_child_weight = 3, reg_lambda=0.5))
]

In [ ]:
model_xgbr = Pipeline(steps = steps_xgbr)
model_xgbr.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric', DataTransformer(),
                                                  ['cost', 'weight', 'year']),
                                                 ('numeric_pca',
                                                  PCATransformer(),
                                                  ['height', 'width', 'depth']),
                                                 ('categorical_product_type',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['product_type']),
                                                 ('categorical_product_level',
                                                  OrdinalEncoder(),
                                                  ['product_level']),
                                                 ('catego...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.05,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=3, max_leaves=None, min_child_weight=3,
                              missing=nan, monotone_constraints=None,
                              multi_strategy=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=None, ...))])

In [ ]:
y_train_pred = model_xgbr.predict(X_train)
print('train MAE: {0:.2e}'.format(mean_absolute_error(y_train, y_train_pred)))
print('train MSE: {0:.2e}'.format(mean_squared_error(y_train, y_train_pred)))
print('train R2: {0:.3f}'.format(r2_score(y_train, y_train_pred)))

y_test_pred = model_xgbr.predict(X_test)
print('test MAE: {0:.2e}'.format(mean_absolute_error(y_test, y_test_pred)))
print('test MSE: {0:.2e}'.format(mean_squared_error(y_test, y_test_pred)))
print('test R2: {0:.3f}'.format(r2_score(y_test, y_test_pred)))

train MAE: 9.57e+04
train MSE: 2.04e+10
train R2: 0.522
test MAE: 1.21e+05
test MSE: 4.47e+10
test R2: 0.482


In [ ]:
model_gsv = GridSearchCV(model_xgbr, parameters, cv = 5, scoring = scorer)
model_gsv = model_gsv.fit(X_train, y_train)

In [ ]:
model_gsv.best_params_

{'xgbr__gamma': 0.0,
 'xgbr__lambda': 0.5,
 'xgbr__learning_rate': 0.05,
 'xgbr__max_depth': 3,
 'xgbr__min_child_weight': 3,
 'xgbr__n_estimators': 100}

In [ ]:
y_train_pred = model_gsv.predict(X_train)
print('train MAE: {0:.2e}'.format(mean_absolute_error(y_train, y_train_pred)))
print('train MSE: {0:.2e}'.format(mean_squared_error(y_train, y_train_pred)))
print('train R2: {0:.3f}'.format(r2_score(y_train, y_train_pred)))

y_test_pred = model_gsv.predict(X_test)
print('test MAE: {0:.2e}'.format(mean_absolute_error(y_test, y_test_pred)))
print('test MSE: {0:.2e}'.format(mean_squared_error(y_test, y_test_pred)))
print('test R2: {0:.3f}'.format(r2_score(y_test, y_test_pred)))

train MAE: 9.57e+04
train MSE: 2.04e+10
train R2: 0.522
test MAE: 1.21e+05
test MSE: 4.47e+10
test R2: 0.482
